# RAG and tatters

Threadbare implementation of RAG.

## setup

In [2]:
%pip install -q ipywidgets html2text langchain langchain_community sentence-transformers 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch as t

from bs4 import BeautifulSoup
import requests
import html2text

### utils

In [34]:
def wikipedia_to_markdown(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})
    return html2text.html2text(str(content))

In [35]:
def scope():
    embeddings = HuggingFaceEmbeddings()
    def get_embedding(text):
        return t.tensor(embeddings.embed_query(text))
    return get_embedding

get_embedding = scope()

## document chunking

One option is to ingest full documents, another is to chunk them into smaller pieces. Let's compare different schemes of document chunking.

In [36]:
document = wikipedia_to_markdown('https://en.wikipedia.org/wiki/Magician_(fantasy)')

### fixed size

Split the text into fixed size chunks. For simplicity I'll fix the size in characters, but it would be smarter to split by amount of tokens instead.

In [37]:
def chunk_fixed_size(text, size):
    return [text[i:i+size] for i in range(0, len(text), size)]

chunks = chunk_fixed_size(document, 100)
chunks[:4]

['Magicians appearing in fantasy fiction\n\nFor other uses, see [Magician\n(disambiguation)](/wiki/Magici',
 'an_\\(disambiguation\\) "Magician\n\\(disambiguation\\)") and [Magi (disambiguation)](/wiki/Magi_\\(disamb',
 'iguation\\)\n"Magi \\(disambiguation\\)").\n\n"Wizard (fantasy)" redirects here. For other uses, see [Wiza',
 'rd\n(disambiguation)](/wiki/Wizard_\\(disambiguation\\) "Wizard\n\\(disambiguation\\)").\n\n[![](//upload.wi']

In [38]:
def chunk_fixed_size_overlap(text, size, overlap):
    return [text[i:i+size] for i in range(0, len(text), size - overlap)]

chunks = chunk_fixed_size_overlap(document, 100, 10)
chunks[:4]

['Magicians appearing in fantasy fiction\n\nFor other uses, see [Magician\n(disambiguation)](/wiki/Magici',
 'iki/Magician_\\(disambiguation\\) "Magician\n\\(disambiguation\\)") and [Magi (disambiguation)](/wiki/Mag',
 '(/wiki/Magi_\\(disambiguation\\)\n"Magi \\(disambiguation\\)").\n\n"Wizard (fantasy)" redirects here. For o',
 'ere. For other uses, see [Wizard\n(disambiguation)](/wiki/Wizard_\\(disambiguation\\) "Wizard\n\\(disambi']

### recursive character split

Split on a hierarchy of specific landmarks (e.g. `'\n\n'`, `'\n'`, `' '`) until we reach the desired size. This is meant to preserve more structure than simple fixed size split.

In [49]:
def chunk_recursive_character_split(text, size, separators=['\n\n', '\n', ' ']):
    if len(text) <= size: return [text]
    for separator in separators + ['']:
        if (index := text[:size].rfind(separator)) != -1:
            index += len(separator)
            return [text[:index]] + chunk_recursive_character_split(text[index:], size, separators)

chunks = chunk_recursive_character_split(document, 100)
chunks[:4]

['Magicians appearing in fantasy fiction\n\n',
 'For other uses, see [Magician\n(disambiguation)](/wiki/Magician_\\(disambiguation\\) "Magician\n',
 '\\(disambiguation\\)") and [Magi (disambiguation)](/wiki/Magi_\\(disambiguation\\)\n',
 '"Magi \\(disambiguation\\)").\n\n']

### document specific splitting

Split based on the document specific grammar (e.g. markdown, HTML, PDF ...).

In [40]:
def chunk_markdown(text, size, offset=0):
    ''' piggyback on recursive character split for the demo but it should use a markdown parser '''
    separators = [
        '\n# ', '\n## ', '\n### ', '\n#### ', '\n##### ', '\n###### ', # headings
        '```\n', '\n\n', # blocks
        '\n', '`', '[', ']', '(', ')', '*', '_', # inline
        ' ', # words
    ]
    if len(text) <= size: return [text]
    for separator in separators + ['']:
        if (index := text[offset:size].rfind(separator)) != -1:
            index += offset
            return [text[:index]] + chunk_markdown(text[index:], size, offset=len(separator))

doc = '''
# The Enigmatic Life of Wizard Eldrath

## Introduction
Eldrath the Wise, a wizard of great renown, has fascinated scholars and adventurers alike with his mysterious powers and secretive nature.

## Notable Achievements
Eldrath is known for many great deeds, including the discovery of the lost city of Aranthar and the creation of the spell of eternal light.
```
def eternal_light():
    while True:
        light_spell.cast()
```
Which remains one of the most powerful enchantments in existence.
'''
chunks = chunk_markdown(doc, 100)
chunks[:10]

['',
 '\n# The Enigmatic Life of Wizard Eldrath\n',
 '\n## Introduction',
 '\nEldrath the Wise, a wizard of great renown, has fascinated scholars and adventurers alike with his',
 ' mysterious powers and secretive nature.\n',
 '\n## Notable Achievements',
 '\nEldrath is known for many great deeds, including the discovery of the lost city of Aranthar and',
 ' the creation of the spell of eternal light.\n',
 '```\ndef eternal_light():\n    while True:\n        light_spell.cast()\n',
 '```\nWhich remains one of the most powerful enchantments in existence.\n']

### semantic splitting

Split the document based on meaning. A naive aproach is to split the document into sentences (here I'll re-use recursive character split) compute their embeddings. Use the embeddings to find topics boundary in the text and merge the rest together.

In [119]:
def cluster_chunks(chunks, indices, size=500):
    ''' cluster chunks such that:
      - each cluster is smaller or equal to size
      - chunks are clustered according to their relative similarities
    '''
    indices = [i + 1 for i in indices] # shift to the right

    def rec(start, end, idx=0):
        # shortcircuit if the entire chunk fits
        if sum(len(c) for c in chunks[start:end]) <= size:
            return [''.join(chunks[start:end])]
        for i in range(idx, len(indices)):
            index = indices[i]
            if start < index < end:
                return rec(start, index, i + 1) + rec(index, end, i + 1)
    
    return rec(0, len(chunks))

def test_cluster_chunks_size():
    chunks = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
    indices = [2, 5, 8, 1, 0, 3, 6, 4, 7]
    assert cluster_chunks(chunks, indices, size=1) == ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
    assert cluster_chunks(chunks, indices, size=2) == ['ab', 'c', 'd', 'ef', 'g', 'hi', 'j']
    assert cluster_chunks(chunks, indices, size=3) == ['abc', 'def', 'ghi', 'j']
    assert cluster_chunks(chunks, indices, size=4) == ['abc', 'def', 'ghij']
    assert cluster_chunks(chunks, indices, size=7) == ['abc', 'defghij']
    assert cluster_chunks(chunks, indices, size=10) == ['abcdefghij']

test_cluster_chunks_size()

In [120]:
def chunk_semantic(text, size=100):
    mini_chunks = chunk_recursive_character_split(text, size)
    embeddings = [get_embedding(c) for c in mini_chunks]
    similarities = t.cosine_similarity(t.stack(embeddings[:-1]), t.stack(embeddings[1:]))
    _, indices = t.sort(similarities)
    return cluster_chunks(mini_chunks, indices)

chunks = chunk_semantic(document)
chunks[:4]

['Magicians appearing in fantasy fiction\n\nFor other uses, see [Magician\n(disambiguation)](/wiki/Magician_\\(disambiguation\\) "Magician\n\\(disambiguation\\)") and [Magi (disambiguation)](/wiki/Magi_\\(disambiguation\\)\n"Magi \\(disambiguation\\)").\n\n',
 '"Wizard (fantasy)" redirects here. For other uses, see [Wizard\n(disambiguation)](/wiki/Wizard_\\(disambiguation\\) "Wizard\n\\(disambiguation\\)").\n\n[![](//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-\nnew.svg/50px-Question_book-new.svg.png)](/wiki/File:Question_book-new.svg)|\n',
 'This article **needs additional citations\n',
 'for[verification](/wiki/Wikipedia:Verifiability "Wikipedia:Verifiability")**.\n']

## document retrieval

In [123]:
# create a dummy database for our embegginds / chunks pairs
def create_db(documents):
    chunks = [chunk for document in documents for chunk in chunk_recursive_character_split(document, 100)]
    db = t.stack([get_embedding(chunk) for chunk in chunks])
    return chunks, db

chunks, db = create_db([document])

### exhaustive search

In [146]:
def retrieve(query, k=3, threshold=0.5):
    query_embedding = get_embedding(query)
    similarities = t.cosine_similarity(db, query_embedding)
    values, indices = t.topk(similarities, k=k)
    indices = indices[values > threshold]
    return [chunks[i] for i in indices]

print(retrieve('dnd'))
print(retrieve('banana'))
print(retrieve('merlin the enchanter'))
print(retrieve('harry potter'))

['  * _[Dungeons& Dragons](/wiki/Dungeons_%26_Dragons "Dungeons & Dragons")_\n', 'the _[Dungeons& Dragons](/wiki/Dungeons_%26_Dragons "Dungeons & Dragons")_\n']
[]
['Pyle_The_Enchanter_Merlin.JPG)_The Enchanter Merlin_ , by [Howard\n', 'Pyle_The_Enchanter_Merlin.JPG/170px-Arthur-\nPyle_The_Enchanter_Merlin.JPG)](/wiki/File:Arthur-\n', '"Mentor"), with [Merlin](/wiki/Merlin "Merlin") from the [_King Arthur_\n']
['series of books by [J. K. Rowling](/wiki/J._K._Rowling "J. K. Rowling").\n\n', 'the Rings_ or [Lord Voldemort](/wiki/Lord_Voldemort "Lord Voldemort") from\n', 'Lord of the Rings](/wiki/The_Lord_of_the_Rings "The Lord of the Rings")_ and\n']


### aproximate nearest neighborgs (ANN)

#### navigable small worlds (NSW)